In [13]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [14]:
xls = pd.ExcelFile('Coronna Data CERTAIN with KVB edits.xlsx')
sheet2 = pd.read_excel(xls, 'BL+3M')
data1 = sheet2.copy()

In [15]:
data1 = data1.drop(columns = ['SubjectID', 'CDate', 'Match'])
data1 = data1.drop(columns = ['SubjectID.1', 'UNMC_id.1', 'CDate.1', 'init_group.1', 'grp.1', 'UNMC_id.2', 'grp.2', 'init_group.2', 'CDate.1', 'futime.1'])
data1 = data1.drop(columns = ['init_group', 'futime'])
#Drop the features with more than 70% NA values (other than smkyrs)
data1 = data1.drop(columns = ['statin_use', 'rfstatus_impute', 'ccpstatus_impute', 'statin_use.1'])
#Drop the features that are considered not usable
data1 = data1.drop(columns = ['nonpresNSAIDs_use', 'NSAIDs_use','nonpresNSAIDs_use.1', 'NSAIDs_use.1'])
#Distinguish different features with the same name
data1 = data1.rename(columns={"usresultsIgA.1": "usresultsIgA_BL", "usresultsIgG.1": "usresultsIgG_BL", "usresultsIgM.1": "usresultsIgM_BL", 'seatedbp1.1': 'seatedbp1_BL',
                              'seatedbp2.1': 'seatedbp2_BL', 'pres_mtx.1': 'pres_mtx_BL', 'pres_arava.1': 'pres_arava_BL', 'pres_azulfidine.1': 'pres_azulfidine_BL', 
                              'pres_plaquenil.1': 'pres_plaquenil_BL', 'pres_imuran.1': 'pres_imuran_BL', 'pres_minocin.1': 'pres_minocin_BL', 'pres_pred.1': 'pres_pred_BL',
                              'statin_use.1': 'statin_use_BL', 'tender_jts_28.1': 'tender_jts_28_BL', 'BLswollen_jts_28': 'swollen_jts_28_BL',
                              'BLmd_global_assess': 'md_global_assess_BL', 'BLpt_global_assess': 'pt_global_assess_BL', 'BLdi': 'di_BL', 'BLpt_pain': 'pt_pain_BL', 'BLusresultsCRP': 'usresultsCRP_BL',
                              'DAS28-CRP BL': 'DAS28-CRP_BL', })
data1['BMI'] = data1['weight'] / (data1['height'] ** 2) * 703 
data1['IFNβ activity final'][234] = None
data1['IFNα activity final'][234] = None
data1['IFNβ/α ratio final'][234] = None

/state/partition1/job-15414968/ipykernel_3981521/3999710629.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['IFNβ activity final'][234] = None
/state/partition1/job-15414968/ipykernel_3981521/3999710629.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['IFNα activity final'][234] = None
/state/partition1/job-15414968/ipykernel_3981521/3999710629.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['IFNβ/α rati

In [18]:
data=data1
data.shape

(296, 80)

In [16]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import impyute
from sklearn.impute import KNNImputer
from sklearn import preprocessing
from sklearn.preprocessing import scale

In [19]:
data = data[data['3MResponse'] != 'Unknown']
def flag_na(row):
    if math.isnan(row['IFNβ/α ratio final']):
        return 1
    else:
        return 0
#This line of code iterate over each row of IFN and add a new feature to indicate whether IFN is missing
data['IFN Missing'] = data.apply(lambda row: flag_na(row), axis = 1)
data = data.fillna({'Type I IFN activity':0, 'IFNβ activity final':0, 'IFNα activity final':0, 'IFNβ/α ratio final':0})
data = data.drop(columns = ['seatedbp1', 'seatedbp2', 'pres_mtx', 'pres_arava', 'pres_azulfidine', 'pres_plaquenil',
                           'pres_pred', 'md_global_assess', 'pt_global_assess', 'di', 'pt_pain', 'usresultsIgA', 
                           'usresultsIgG', 'usresultsIgM'])

#pres_imuran, pres_minocin, pres_minocin_BL, num_tnf, num_nontnf are all zeros
#This kind of variables will not provide useful information
data = data.drop(columns = ['pres_imuran', 'pres_minocin', 'pres_minocin_BL', 'num_tnf', 'num_nontnf'])
#these are the columns that have correlation with the target feature
#We will need to delete these values to avoid redundent information and noise
data = data.drop(columns = ['ara_func_class', 'tender_jts_28', 'swollen_jts_28', 'usresultsCRP'])
data = data[data['drinker'].notnull()]
#Because it does not have 0 value for normal data, thus here zero will indicate it has a missing value rather than a actual number
data = data[data['usresultsRF'].notnull()]
#Because it does not have 0 value for normal data, thus here zero will indicate it has a missing value rather than actual number
data['usresultsCCP3'] = data['usresultsCCP3'].replace(np.nan, 0)
#Drops the rows with missing value for seatedbp1_BL
data = data[data['seatedbp1_BL'].notnull()]
data = data[data['md_global_assess_BL'].notnull()]
data = data.fillna({'usresultsIgA_BL': 0, 'usresultsIgG_BL':0, 'usresultsIgM_BL':0})
data = data[data['BMI'].notnull()]
data = data.drop(columns = ['UNMC_id'])
data = data[data['final_education'].notnull()]
data = data[data['final_education'] != "don't remember"]
data = data.drop(columns = ['smkyrs', 'numcigs'])

In [7]:
encoders = dict()

for col_name in ['3MResponse', 'grp', 'gender', 'final_education', 'race_grp', 'newsmoker', 'drinker']:
    series = data[col_name]
    label_encoder = preprocessing.LabelEncoder()
    data[col_name] = pd.Series(
        label_encoder.fit_transform(series[series.notnull()]),
        index=series[series.notnull()].index
    )
    encoders[col_name] = label_encoder
data3 = data.copy()
data3 = data3[data3['ethnicity'].notnull()]
data3 = data3[data3['newsmoker'].notnull()]


In [8]:
data3.to_csv('Analytical_Base_Table_No_KNN.csv', index = False)

In [11]:
data

,3MResponse,grp,Type I IFN activity,IFNβ activity final,IFNα activity final,IFNβ/α ratio final,DAS28-CRP 3m,age,gender,final_education,race_grp,ethnicity,weight,height,newsmoker,drinker,drinksperwk,duration_ra,ndmardused,rfstatus_b,usresultsRF,ccpstatus_b,usresultsCCP3,hxmi,hxunstab_ang,hxchf,hxother_cv,hxtia,hxstroke,hxcopd,hxhtn,hxdiabetes,hxhld,hx_anycancer,seatedbp1_BL,seatedbp2_BL,pres_mtx_BL,pres_arava_BL,pres_azulfidine_BL,pres_plaquenil_BL,pres_imuran_BL,pres_pred_BL,tender_jts_28_BL,swollen_jts_28_BL,md_global_assess_BL,pt_global_assess_BL,di_BL,pt_pain_BL,usresultsCRP_BL,DAS28-CRP_BL,usresultsIgA_BL,usresultsIgG_BL,usresultsIgM_BL,BMI,IFN Missing
0,0,0,0.000000,0.000000,0.000000,0.000000,1.249516,62,1,0,6,1.0,219.0,74,1.0,1,7.000000,1,1,0,8.7,0,16.0,0,0,0,0,0,0,0,0,0,0,0,110.0,71.0,1,0,0,0,0,0,5,3,35.0,10.0,0.000000,10.0,4.41,3.444942,284.0,1640.0,94.0,28.114865,1
1,2,1,0.814954,0.714954,0.714954,1.000000,2.473876,54,0,0,6,1.0,175.0,66,2.0,1,2.000000,2,1,1,16.8,0,15.0,0,0,0,0,0,0,0,0,0,0,0,108.0,69.0,0,0,0,0,0,0,3,3,35.0,17.0,0.250000,25.0,1.19,2.935127,207.0,1010.0,56.0,28.242654,0
2,1,1,4.582632,2.869955,4.482632,0.640239,3.386822,63,0,0,6,1.0,117.0,64,0.0,1,35.000000,1,1,0,11.3,1,20.0,0,0,0,0,0,0,0,0,0,1,0,128.0,76.0,1,0,0,1,0,0,2,6,40.0,10.0,0.000000,5.0,0.40,2.698947,190.0,657.0,100.0,20.080811,0
3,1,1,0.015811,0.000000,0.000000,0.000000,4.444948,61,0,0,6,1.0,240.0,65,1.0,1,0.466667,1,1,0,9.8,0,15.0,0,0,0,0,0,0,0,0,1,0,0,110.0,80.0,1,0,0,0,0,1,3,9,55.0,50.0,0.250000,60.0,1.00,3.719481,44.0,948.0,51.0,39.933728,1
4,0,2,0.683486,0.583486,0.583486,1.000000,3.143985,51,0,1,6,1.0,90.0,59,0.0,0,0.000000,5,1,1,30.5,1,174.0,0,0,0,0,0,0,0,0,0,0,0,100.0,70.0,1,0,0,0,0,1,14,13,65.0,20.0,0.250000,65.0,0.25,4.425214,155.0,824.0,43.0,18.175812,0
6,0,2,0.000000,0.000000,0.000000,0.000000,1.960279,62,0,0,6,1.0,150.0,63,1.0,1,14.000000,2,1,0,9.4,0,15.0,0,0,0,0,0,0,0,1,0,0,0,112.0,75.0,1,0,0,0,0,0,12,12,50.0,25.0,0.000000,20.0,17.93,5.278515,171.0,992.0,95.0,26.568405,1
7,1,2,0.484360,0.100000,0.384360,0.260173,2.319729,35,0,0,6,1.0,211.0,64,1.0,1,7.000000,11,3,0,6.0,0,15.0,0,0,0,0,0,0,0,0,0,0,0,112.0,78.0,0,1,0,0,0,1,1,1,25.0,55.0,0.625000,60.0,3.60,3.119380,219.0,1160.0,138.0,36.214111,0
8,0,1,0.000000,0.000000,0.000000,0.000000,1.532090,54,1,1,6,1.0,182.0,71,0.0,1,0.466667,2,1,1,750.0,1,251.0,0,0,0,0,0,0,0,0,0,0,0,119.0,76.0,1,0,0,0,0,0,5,5,40.0,10.0,0.142857,5.0,4.70,3.604865,267.0,1110.0,109.0,25.381075,1
9,2,1,0.000000,0.000000,0.000000,0.000000,3.370891,50,0,0,6,1.0,250.0,67,1.0,1,0.466667,0,1,1,92.4,1,73.0,0,0,0,0,0,0,0,0,0,0,0,110.0,80.0,0,0,0,0,0,0,7,6,35.0,27.0,0.500000,20.0,1.33,3.809990,402.0,1290.0,89.0,39.151259,1
10,1,2,1.836340,1.736340,1.736340,1.000000,1.648763,54,1,0,6,1.0,178.0,66,1.0,1,14.000000,0,0,0,7.8,0,15.0,0,0,0,0,0,0,0,0,0,0,0,120.0,76.0,0,0,0,0,0,0,4,3,40.0,10.0,0.000000,25.0,0.62,2.878648,213.0,1050.0,83.0,28.726814,0


In [9]:
#This is a built in algorithm to built KNN model for Imputation
#n_neighbors controls the number of nearest points to be considered when calculating similarity
imputer = KNNImputer(n_neighbors=50, weights="uniform")
data_temp = imputer.fit_transform(data)
data_temp = pd.DataFrame(data_temp, columns = data.columns)
data_temp.to_csv('Analytical_Base_Table.csv', index = False)

In [10]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import confusion_matrix
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data_temp = pd.read_csv('Analytical_Base_Table.csv')
#Because we wanted to use regression later, we kepted these two columns in Analytical Base Table
#Thus, now we need to drop them for the classification
data4 = data_temp.drop(columns = ['DAS28-CRP 3m', 'DAS28-CRP_BL'])
data4 = data4.drop(columns = ['weight', 'height'])


In [11]:
X = data4.drop('3MResponse',1)                            ### Dividing the dataframe into X and y set
y = data4['3MResponse']

/state/partition1/job-15128777/ipykernel_649704/2884243457.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = data4.drop('3MResponse',1)                            ### Dividing the dataframe into X and y set


In [12]:
from sklearn.preprocessing import LabelEncoder
y = LabelEncoder().fit_transform(y.astype('str'))

In [13]:
X = X.drop(columns=['hxunstab_ang'])

In [14]:
labels = ['grp','gender','final_education','race_grp','newsmoker','drinker','ara_func_class']     ### normalizing the dataset

for i in X.columns:
    if i not in labels:
        mean = X[i].mean()
        std = X[i].std()
        X[i] = (X[i] - mean) / std

In [15]:
X['y'] = y
X.to_csv('preprocessing_approach_2.csv', index = False, )

#X_train_norm, X_val_norm, Y_train, Y_val = train_test_split(X, y, random_state=1337, test_size=0.1)

#for i in X_train_norm.columns:           ### Checking the null values
    print(i, X_val_norm[i].isnull().sum())

<!-- #import imblearn                                      ### sampling the data using the smote method
#from imblearn.over_sampling import RandomOverSampler
#from imblearn.over_sampling import SMOTE
#def sampling(over_under,X,y):
    if over_under == 'under':
        sampler = imblearn.under_sampling.RandomUnderSampler(random_state = rs)
        X_under, y_under = sampler.fit_resample(X, y)
        return X_under,y_under
    elif over_under == 'over':
        ros = RandomOverSampler(random_state=1337)
        X_over, y_over = ros.fit_resample(X, y)
        return X_over,y_over
    else:
        sampler = SMOTE()
        X_smote, y_smote = sampler.fit_resample(X, y)
        return X_smote,y_smote
#print(X)
#print(y)
X_final,y_final = sampling('smote',X_train_norm,Y_train) -->

<!-- import sklearn
model3 = None
def random_forest(X_train_norm, X_test, Y_train,Y_test):
    global model3
    print(model3)
    model3 = sklearn.ensemble.RandomForestClassifier(oob_score = True, random_state=1337)
    model3.fit(X_train_norm, Y_train)
    print(model3.oob_score_)
    print(f"Training accuracy is {model3.score(X_train_norm, Y_train)}")
    print(f"Testing accuracy is {model3.score(X_test, Y_test)}")
    #sklearn.metrics.plot_confusion_matrix(model3, X_test, Y_test)
    #sklearn.metrics.plot_roc_curve(model3, X_test, Y_test)
random_forest(X_final, X_val_norm, y_final, Y_val) -->

<!-- y = data1['3MResponse']
X = data1.drop(columns = ['3MResponse'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
clf = RandomForestClassifier(random_state = 42)
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)
print(accuracy_score(y_test, prediction)) -->